In [1]:
import readers
import text_chunks
import os
import pandas as pd

from dotenv import load_dotenv, find_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain 
from langchain.chains import LLMChain
from langchain.schema.messages import HumanMessage
from langchain.callbacks import get_openai_callback

In [2]:
# check API
load_dotenv(find_dotenv())

True

In [3]:
# queries
query_name = "query1"
query = readers.read_txt(f'query/{query_name}.txt')

In [4]:
# GLOVAL LLM VALUES
MODEL_NAME = 'gpt-3.5-turbo'
MODEL_NAME_16k = 'gpt-3.5-turbo-16k'
TEMPERATURE = 0.0

# Define llm
llm = ChatOpenAI(model=MODEL_NAME, temperature=TEMPERATURE)

In [5]:
# Ppepare templates
STANDART_TEMPLATE = """
Ти викладач юридичного права України.
Ти намагаєшся відповідати на питання студентів наступним шаблоном:
"[Імʼя особи]-[пункт, частина, статаття, що передбачає покарання за вчинене кримінальне правопорушення, назва статті]."
Статей може бути декілька.]"
Ти використовуєш кримінальний кодекс України.
Якщо відповіді ти не знаєш, відповідай "Не певен, але вважаю що" і намагайся здогадатися, якщо знаєш відповідай "Відповідь така".
{query}
"""
LIMITED_ARTICLES_TEMPLATE = """
Ти спеціаліст по праву. 
Якщо відповіді ти не знаєш, відповідай "Не певен, але вважаю що" і намагайся здогадатися, якщо знаєш відповідай "Відповідь така" 
{query} Якими статтями можна описати діяння учасників?
Стаття 124. Умисне заподіяння тяжких тілесних ушкоджень у разі перевищення меж 
Стаття 125. Умисне легке тілесне ушкодження
Стаття 126. Побої і мордування
Стаття 126-1. Домашнє насильство
Стаття 292. Пошкодження об'єктів магістральних або промислових нафто-, газо-, конденсатопроводів та нафтопродуктопроводів
Стаття 293. Групове порушення громадського порядку
Стаття 300. Ввезення, виготовлення або розповсюдження творів, що пропагують культ 
Стаття 322. Незаконна організація або утримання місць для вживання одурманюючих засобів
Стаття 323. Спонукання неповнолітніх до застосування допінгу
Стаття 324. Схиляння неповнолітніх до вживання одурманюючих засобів
"""

In [6]:
# Define prompt
prompt_template = PromptTemplate(input_variables=['query'], 
                                 template=LIMITED_ARTICLES_TEMPLATE)

print(prompt_template.format(query=query))

# Count the query tokens
with get_openai_callback() as cb:
    result = llm([HumanMessage(content=prompt_template.format(query=query))])
    print(f"\n Total Tokens: {cb.total_tokens}")
    print(f" Prompt Tokens: {cb.prompt_tokens}")
    print(f" Completion Tokens: {cb.completion_tokens}")
    print(f" Total Cost (USD): ${round(cb.total_cost, 3)}")



Ти спеціаліст по праву. 
Якщо відповіді ти не знаєш, відповідай "Не певен, але вважаю що" і намагайся здогадатися, якщо знаєш відповідай "Відповідь така" 
Контекст: Вахтанг, керівник однієї з політичних партій в Україні, перебуваючи у відрядженні у Республіці Польща, дізнався, що його громадянство України було припинено з підстав неналежного оформленні документів під час одержання такого громадянства. Бажаючи продовжувати свою політичну діяльність, Вахтанг домовився із Василем, Петром та Іваном, що ті організують «прихильників», які створять «масовку» під час переходу ним державного кордону України зі сторони Республіки Польща.
11 вересня 2021 року Вахтанг прибув потягом до одного з пунктів пропуску на польсько-українському кордоні, зійшов з нього та у оточенні своїх прихильників (в кількості близько 20 осіб) яких покликали Василь та Петро, почав рухатися до пішохідного переходу. Іван та близько 200 прихильників Вахтанга чекали його на території України, викрикуючи політичні гасла. По

In [7]:
# define chain
chain = LLMChain(llm=llm, prompt=prompt_template, verbose=False)

In [8]:
response = chain.run(query=query)

In [11]:
# Report
report = f"""
The answer was generated by {MODEL_NAME}, temperature={TEMPERATURE}:\n***\n{response}\n***\n
"""
print(report)


The answer was generated by gpt-3.5-turbo, temperature=0.0:
***
Стаття 125. Умисне легке тілесне ушкодження - оскільки 11 працівників Національної поліції та 5 працівників ДПС України одержали легкі тілесні ушкодження під час штовханини і бійок.
Стаття 293. Групове порушення громадського порядку - оскільки виникла штовханина та бійки між прихильниками Вахтанга та службовцями ДПС, а також черга з автомобілів та громадян.
Можливо також можна розглядати статтю 126. Побої і мордування, оскільки прихильники Вахтанга відштовхували службовців ДПС, створюючи "живий коридор". Однак, для точної кваліфікації цього діяння потрібно більше деталей.
***




In [14]:
# Save response
from datetime import date

num = 0
save_path = "response/"

while os.path.exists(f"{save_path}{query_name}_{date.today()}_({num}).txt"):
    num += 1

file_name = f"{query_name}_{date.today()}_({num}).txt"
path = save_path+file_name

with open(path, "w", encoding="UTF-8") as save_response:
    result = prompt_template.format(query=query) + report
    save_response.write(result)
save_response.close()